# Parte 1 -  Features Numéricas

# Objetivos:

O objetivo desse desafio nesta primeira etapa é analisar os dados do case e estruturar uma Feature Engineering básica apenas com os dados numéricos existentes, sem transformar ou combinar features ou mesmo adicionar informações externas. 

Ao final do desafio, será treinado um modelo de regressão linear com as features obtidas. Esse modelo será testado contra uma massa de teste, separada previamente.


# Setup do Ambiente

## Magic Functions do Jupyter

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

## Imports de Libs Externas (padrão)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import numpy as np
import os
import pandas as pd

In [ ]:
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.externals import joblib

## Imports de Libs Locais

In [ ]:
from dataset import load_california_housing_prices
from pipeline import NumericalFeaturesImputer, LogFeaturesTransform, FeaturesChoiceTransform
from utils import calculate_outliers

## Carregando o Dataset:

In [ ]:
dataset = load_california_housing_prices()
x_train = dataset["train"]["x"]
y_train = dataset["train"]["y"]
x_test = dataset["test"]["x"]
y_test = dataset["test"]["y"]

# Feature Engineering c/ Features Numéricas

Features numéricas são as primeiras a serem tratadas, por serem as mais fáceis de compreender e de relacionar com o problema. As seções a seguir focam no tratamento e limpeza dessas features, o primeiro passo na engenharia de features.


## Distribuição das Features

In [ ]:
numerical_cols = [
    "longitude", "latitude", 
    "housing_median_age", 
    "total_rooms", "total_bedrooms", 
    "population", "households", "median_income"
]

In [ ]:
# descrevendo as distribuições
cuts = [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]
x_train[numerical_cols].describe(percentiles=cuts)

Para tomada de decisão, é sempre mais informativo usar visualizações. Com elas é possível observar o formato da distribuição (normal, exponencial, logarítmica, etc) e os outliers.

Verificando as distribuições com histogramas:

In [ ]:
_ = x_train[numerical_cols].hist(bins=50, figsize=(15,10))

## Detecção e Tratamento de Nulos

Conhecendo a distribuição das features, já é possível traçar estratégias de tratamento de valores nulos. A primeira tarefa é detectar a proporção de nulos nos dados.


* Verificar quantidade de Nulos em cada feature
* Entender que Nulos podem aparecer também nos dados de teste, portanto apenas descartar o dado não resolve
* Entender que medida (média ou mediana) deve ser imputada em cada feature

In [ ]:
x_null = x_train[numerical_cols].isnull()
null_data = pd.DataFrame({
    "count": x_null.sum(),
    "mean": x_null.mean()
})
null_data

Normalmente a ocorrência de valores nulos ocorre por algum ruído no processo de obtenção de dados, o que significa que muito provavelmente eles ocorrerão também em dados de produção.

Existem algumas técnicas de tratamento de valores nulos:

- Atribuir um valor padrão fora da distribuição;
- Criar modelos para inferir os valores a partir das outras features;
- Imputar um valor referente à distribuição:
    - média
    - mediana
    
Para features numéricas em que conhecemos a distribuição mas não foram tratados os outliers, o mais recomendado é utilizar a **mediana**, cujo valor é pouco afetado por outliers.

--------------
#### Tarefa (1.1) 

Completar a implementação do transformador de dados `NumericalFeaturesImputer` usando a `mediana`. 

A classe está no arquivo `pipeline.py`.

---------------

Pode-se observar a seguir uma amostra dos dados originais, i.e. antes da transformação do imputer. É interessante notar os valores de `NaN` nas colunas `population` e `median_income`.

In [ ]:
sample_x_tr = x_train.head(10).append(x_train.tail(10))
sample_x_tr

Treina-se o `imputer` como se fosse um modelo, usando a massa de teino. A aplicação do `imputer` treinado sobre a amostra está mostrada a seguir. 

In [ ]:
numerical_imputer = NumericalFeaturesImputer(numerical_cols)
numerical_imputer.fit(x_train)
numerical_imputer.transform(sample_x_tr)

Aplicando o `imputer` em toda a massa de treino, pode-se ver o efeito sobre a ocorrência de `NULLs`.

In [ ]:
x_null = numerical_imputer.transform(x_train)[numerical_cols].isnull()
null_data = pd.DataFrame({
    "count": x_null.sum(),
    "mean": x_null.mean()
})
null_data

Para as próximas etapas, será necessário que a massa de treino seja transformada com o `imputer` criado, pois assim não se propaga o problema de haverem valores faltantes na base.

In [ ]:
x_train = numerical_imputer.transform(x_train)

## Transformação Logarítmica de Features

Com os histogramas, já fica evidente que a distribuição de algumas features é **exponencial**. A análise de outliers desse tipo de distribuição pode ser prejudicada pela alta concentração de elementos em uma pequena parte do domínio. 

Uma forma de corrigir essa distorção é transformar esses dados com a função **logarítmica**; a transformação com essa função torna a distribuição das features mais próxima da normal. Por isso, serão transformadas apenas as features que possuem a distribuição exponencial bem evidente.

In [ ]:
log_cols = ["total_rooms", "total_bedrooms", "population", "households", "median_income"]

Ainda é cedo para descartar as features originais tratadas, pois elas podem ter ainda algum poder preditivo que pode ficar ocluso pela transformação. O descarte de features normalmente é feito quando se detecta multi-colinearidade entre as features ou durante uma etapa de _feature selection_.

Por hora, as novas features serão integradas ao dataset original.

--------------
#### Tarefa (1.2) 

Completar a implementação do transformador de dados `LogFeaturesTransform`. 

A classe está no arquivo `pipeline.py`.

---------------

In [ ]:
x_train = LogFeaturesTransform(log_cols).fit_transform(x_train)

In [ ]:
new_cols = [f"log_of_{c}" for c in log_cols]

In [ ]:
_ = x_train[new_cols].hist(bins=50, figsize=(15,10))

## Detecção e Remoção de Outliers

Outliers podem deformar a percepção do domínio para o aprendizado de um modelo linear, impedindo o mesmo de encontrar uma solução correta. 

### Verificando os Box Plots para observar os Outliers

Uma maneira de se estudar os outliers para definir cortes é usando o gráfico `BoxPlot`. 

In [ ]:
plt.figure(figsize=(15,10))
_ = sns.boxplot(x="value", y="variable", data=x_train[new_cols].melt())

### Aplicando os cortes:

Os cortes devem ser aplicados na massa de treino, para que apenas os dados dentro da distribuição correta sejam usados para o treinamento.

--------------
#### Tarefa (1.3)

Usando as distribuições de dados vistas anteriormente, escolher `features` e cortes de mínimo e máximo para completar a implementação dda função `remove_outliers`. 

A classe está no arquivo `utils.py`.

---------------

Na tabela a seguir pode-se observar quais cortes serão aplicados na massa de treino e qual o percentual dos dados que é perdido no processo. 

In [ ]:
keep_index, cuts_table = calculate_outliers(x_train)
ori_size = keep_index.shape[0]
new_size = keep_index.sum()

print(f"Size of 'x_train' before Cuts:\t {ori_size}")
print(f"Size of 'x_train'  after Cuts:\t {new_size} (-{100. * (1. - keep_index.mean()): 0.2f} %)")
cuts_table

Os índices calculados devem ser guardados para uso posterior. 

In [ ]:
joblib.dump(
    value=keep_index,
    filename=os.path.join("pipelines", "keep_index.pkl")
)

## Treinamento e Avaliação de um Modelo Linear

O modelo de machine learning é apenas a parte final de um pipeline de processamentos; o pipeline completo é formado por todas as etapas de pré-processamento desde o dado bruto até as etapas de normalização e redução de dimensionalidade, finalizado pelo modelo preditivo.

O framework `Scikit-Learn` implementa uma ferramenta que permite a montagem de um pipeline completo, que pode ser treinado e usado para predição como um objeto único, que pode ser inclusive salvo em um arquivo. Isso permite que todo o pipeline possa ser exportado para produção sem ser reimplementado.

###  Reload das massas de Treino e de Teste

As massas de dados de Treino e de Teste serão carregadas novamente para que seja aplicado o pipeline de pré-processamento em ambos desde o princípio. 

In [ ]:
dataset = load_california_housing_prices()
x_train = dataset["train"]["x"]
y_train = dataset["train"]["y"]
x_test = dataset["test"]["x"]
y_test = dataset["test"]["y"]

Deve-se remover os outliers da massa de treino usando a função construída para isso.

In [ ]:
keep_index = joblib.load(os.path.join("pipelines", "keep_index.pkl"))
x_train = x_train[keep_index]
y_train = y_train[keep_index]

###  Pipeline de Regressão

A construção do pipeline deve incluir:

* Todas as etapas de pré-processamento
* Seleção de Features
* Normalização (Z-Score)
* Modelo Linear

####  Pipeline de Pré-Processamento

Apenas as etapas da Feature Engineering Numérica devem estar aqui.

In [ ]:
numerical_features = [
    "longitude", "latitude", 
    "housing_median_age", 
    "total_rooms", "total_bedrooms",  
    "population", "households", "median_income"
]

log_transform_features = [
    "total_rooms", "total_bedrooms", 
    "population", "households", 
    "median_income"
]

In [ ]:
feat_eng_pipeline = Pipeline([
    ("numerical_imputer",          NumericalFeaturesImputer(numerical_features)),
    ("logarithmic_transform",      LogFeaturesTransform(log_transform_features))
])

Esse pipeline será treinado e salvo para ser usado nos próximos notebook. 

In [ ]:
feat_eng_pipeline.fit(x_train)
joblib.dump(
    value=feat_eng_pipeline.fit(x_train),
    filename=os.path.join("pipelines", "numerical_feat_eng.pkl")
)

####  Pipeline Completo

Todas as etapas de pré-processamento devem estar incluídas nesse pipeline.

In [ ]:
chosen_features = numerical_features + [f"log_of_{c}" for c in log_transform_features]

In [ ]:
pipeline = Pipeline([
    ("numerical_feat_eng",         feat_eng_pipeline),
    ("features_choice",            FeaturesChoiceTransform(chosen_features)),
    ("zscore",                     StandardScaler()),
    ("predictor",                  ElasticNet()),
])

###  Treinar e avaliar o modelo

Treinamento de todo o pipeline

In [ ]:
pipeline.fit(x_train, y_train)

Avaliação do modelo nas massas de **treino** e de **teste**.

In [ ]:
y_true = y_train
y_pred = pipeline.predict(x_train)
mse_tr = mean_squared_error(y_true=y_true, y_pred=y_pred)
r2_tr = r2_score(y_true=y_true, y_pred=y_pred)

In [ ]:
y_true = y_test
y_pred = pipeline.predict(x_test)
mse_te = mean_squared_error(y_true=y_true, y_pred=y_pred)
r2_te = r2_score(y_true=y_true, y_pred=y_pred)

In [ ]:
pd.DataFrame(
    index=["train", "test"],
    columns=["MSE", "R^2"],
    data=[
        [mse_tr, r2_tr],
        [mse_te, r2_te]
    ]
)